In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import pairwise_distances, mean_absolute_error, r2_score
from sklearn.linear_model import RidgeCV
import optuna
import warnings


warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df = pd.read_csv('train.csv')

print(f'Total rows: {len(df)}')
print(f'There is {df.stock_id.nunique()} unique stocks in the dataset.')
print(f'There is {df.date_id.nunique()} unique dates in the dataset.')

nan_count = df['target'].isna().sum()
print(f"The 'target' column has {nan_count} NaN values.")

print('Null values per column:')
print(df.isnull().sum().sort_values(ascending=False))


Total rows: 5237980
There is 200 unique stocks in the dataset.
There is 481 unique dates in the dataset.
The 'target' column has 88 NaN values.
Null values per column:
far_price                  2894342
near_price                 2857180
ask_price                      220
imbalance_size                 220
reference_price                220
matched_size                   220
wap                            220
bid_price                      220
target                          88
time_id                          0
ask_size                         0
stock_id                         0
bid_size                         0
date_id                          0
imbalance_buy_sell_flag          0
seconds_in_bucket                0
row_id                           0
dtype: int64


In [ ]:
# ---------------------- Memory Optimization ---------------------- #
def reduce_mem_usage(df, verbose=True):
    df = df.copy()
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type.kind not in 'iuf':
            continue
        c_min, c_max = df[col].min(), df[col].max()
        if col_type.kind == 'i':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
        else:
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(f'Memory reduced by {(start_mem - end_mem):.2f} MB ({100 * (start_mem - end_mem)/start_mem:.1f}%)')
    return df

# ---------------------- Feature Engineering ---------------------- #
def feature_engineering(df):
    df = df.copy()
    df['spread'] = df['ask_price'] - df['bid_price']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['price_distance'] = df['reference_price'] - df['mid_price']
    df['spread_ratio'] = df['spread'] / (df['mid_price'] + 1e-6)
    df['wap_relative'] = (df['wap'] - df['reference_price']) / (df['reference_price'] + 1e-6)
    df['liquidity_ratio'] = df['matched_size'] / (df['matched_size'] + df['imbalance_size'] + 1e-6)
    df['bid_pressure'] = df['bid_size'] / (df['bid_size'] + df['ask_size'] + 1e-6)
    df['volume'] = df['ask_size'] + df['bid_size']
    df['size_imbalance'] = df['bid_size'] / (df['ask_size'] + 1e-6)
    df['total_imbalance'] = df['imbalance_size'] * df['imbalance_buy_sell_flag']
    df['price_momentum'] = df['reference_price'] / (df['wap'] + 1e-6)
    df['market_heat'] = df['volume'] / (df['matched_size'] + 1e-6)
    df['far_price'] = df['far_price'].fillna(df['reference_price'])
    df['near_price'] = df['near_price'].fillna(df['reference_price'])
    df['auction_efficiency'] = (df['far_price'] - df['near_price']) / (df['reference_price'] + 1e-6)
    df['price_discovery'] = (df['reference_price'] - df['near_price']) / (df['reference_price'] + 1e-6)
    return df

# ---------------------- Time Features ---------------------- #
def create_time_features(df):
    df = df.copy()
    df['time_to_close'] = 600 - df['seconds_in_bucket']
    df['is_early_auction'] = (df['seconds_in_bucket'] < 300).astype(np.int8)
    df['is_late_auction'] = (df['seconds_in_bucket'] >= 300).astype(np.int8)
    df['seconds_sin'] = np.sin(2 * np.pi * df['seconds_in_bucket'] / 600)
    df['seconds_cos'] = np.cos(2 * np.pi * df['seconds_in_bucket'] / 600)
    return df

# ---------------------- Stock-Level Aggregations ---------------------- #
def compute_stock_level_stats(train_df):
    """Compute and store stock-level statistics from the training data."""
    cols = ['wap', 'imbalance_size', 'matched_size', 'volume', 'spread']
    stock_stats = train_df.groupby('stock_id', observed=True)[cols].agg(['mean', 'std'])
    stock_stats.columns = ['_'.join(c) for c in stock_stats.columns]
    stock_stats = stock_stats.fillna(0)
    return stock_stats

def apply_stock_level_features(df, stock_stats):
    """Use precomputed stock-level stats (from train) on new data."""
    df = df.merge(stock_stats, on='stock_id', how='left')
    for col in ['wap', 'imbalance_size', 'matched_size', 'volume', 'spread']:
        df[f'{col}_stock_normalized'] = (
            df[col] - df[f'{col}_mean']
        ) / (df[f'{col}_std'] + 1e-6)
    return df

# ---------------------- Market-Wide Features ---------------------- #
def create_market_wide_features(df):
    df = df.copy()
    market_stats = df.groupby(['date_id', 'seconds_in_bucket'], observed=True).agg({
        'wap': ['mean', 'std', 'median'],
        'volume': ['mean', 'sum'],
        'imbalance_size': ['mean', 'sum'],
        'bid_size': 'sum',
        'ask_size': 'sum',
        'spread': 'mean'
    }).reset_index()
    market_stats.columns = [
        'date_id', 'seconds_in_bucket',
        'market_wap_mean', 'market_wap_std', 'market_wap_median',
        'market_volume_mean', 'market_volume_total',
        'market_imbalance_mean', 'market_imbalance_total',
        'market_bid_total', 'market_ask_total', 'market_spread_mean'
    ]
    df = df.merge(market_stats, on=['date_id', 'seconds_in_bucket'], how='left')
    df['market_sentiment'] = df['market_bid_total'] / (df['market_bid_total'] + df['market_ask_total'] + 1e-6)
    df['market_imbalance_ratio'] = df['market_imbalance_total'] / (df['market_volume_total'] + 1e-6)
    df['market_volatility'] = df['market_wap_std'] / (df['market_wap_mean'] + 1e-6)
    df['wap_vs_market'] = df['wap'] / (df['market_wap_mean'] + 1e-6)
    df['volume_vs_market'] = df['volume'] / (df['market_volume_mean'] + 1e-6)
    return df

# ---------------------- Lag Features ---------------------- #
def create_same_day_lag_features(df, lag_steps=[1,3,6]):
    df = df.sort_values(['stock_id', 'date_id', 'seconds_in_bucket']).reset_index(drop=True)
    grouped = df.groupby(['stock_id','date_id'], observed=True)
    lag_cols = ['wap', 'wap_relative', 'liquidity_ratio', 'bid_pressure',
                'imbalance_size', 'matched_size', 'volume', 'spread']
    eps = 1e-9
    for col in lag_cols:
        for step in lag_steps:
            lag_col = f"{col}_lag_{step}"
            prev = grouped[col].shift(step)
            df[lag_col] = (df[col] - prev) / (prev + eps)
            df[lag_col] = df[lag_col].fillna(0)
    return df




# ---------------------- Extra Features ---------------------- #
def extra_features(df):
    df = df.copy()
    eps = 1e-9
    # immediate changes
    df = df.sort_values(['stock_id', 'date_id', 'seconds_in_bucket'])
    df['wap_prev'] = df.groupby(['stock_id','date_id'])['wap'].shift(1)
    df['wap_diff'] = (df['wap'] - df['wap_prev']).fillna(0)
    df['wap_ret'] = df['wap_diff'] / (df['wap_prev'] + eps)
    # near/far gaps
    df['near_far_gap'] = df['far_price'] - df['near_price']
    df['near_gap_rel'] = (df['near_price'] - df['reference_price']) / (df['reference_price'] + eps)
    df['far_gap_rel'] = (df['far_price'] - df['reference_price']) / (df['reference_price'] + eps)
    # logs
    df['log_matched'] = np.log1p(df['matched_size'])
    df['log_imbalance'] = np.log1p(np.abs(df['imbalance_size']))
    # signed imbalance already present as total_imbalance
    # rank within timestamp
    df['wap_rank'] = df.groupby(['date_id','seconds_in_bucket'])['wap'].rank(pct=True)
    df['spread_rank'] = df.groupby(['date_id','seconds_in_bucket'])['spread'].rank(pct=True)
    df['liquidity_rank'] = df.groupby(['date_id','seconds_in_bucket'])['matched_size'].rank(pct=True)
    # simple EWMA within day (past only)
    df['wap_ewm_3'] = df.groupby(['stock_id','date_id'])['wap'].apply(lambda x: x.shift(1).ewm(span=3, adjust=False).mean()).reset_index(level=[0,1], drop=True)
    df['wap_ewm_5'] = df.groupby(['stock_id','date_id'])['wap'].apply(lambda x: x.shift(1).ewm(span=5, adjust=False).mean()).reset_index(level=[0,1], drop=True)
    df.fillna(0, inplace=True)
    return df




# ---------------------- Pipeline ---------------------- #
def process_dataset(train_df, test_df=None):
    print("▶ Step 1: Basic feature engineering...")
    train_df = feature_engineering(train_df)
    if test_df is not None:
        test_df = feature_engineering(test_df)

    print("▶ Step 2: Time features...")
    train_df = create_time_features(train_df)
    if test_df is not None:
        test_df = create_time_features(test_df)

    print("▶ Step 3: Compute stock-level features from train and apply to all...")
    stock_stats = compute_stock_level_stats(train_df)
    train_df = apply_stock_level_features(train_df, stock_stats)
    if test_df is not None:
        test_df = apply_stock_level_features(test_df, stock_stats)

    print("▶ Step 4: Market-wide features...")
    train_df = create_market_wide_features(train_df)
    if test_df is not None:
        test_df = create_market_wide_features(test_df)

    print("▶ Step 5: Lag features...")
    train_df = create_same_day_lag_features(train_df)
    if test_df is not None:
        test_df = create_same_day_lag_features(test_df)

    print("▶ Step 6: Extra features...")
    train_df = extra_features(train_df)
    if test_df is not None:
        test_df = extra_features(test_df)


    # Handle missing values

    num_cols = train_df.select_dtypes(include=[np.number]).columns
    num_cols = [c for c in num_cols if c != 'target']

    # Fill missing values in train and test using train medians
    train_df[num_cols] = train_df[num_cols].fillna(train_df[num_cols].median())
    if test_df is not None:
        common_cols = [c for c in num_cols if c in test_df.columns]
        test_df[common_cols] = test_df[common_cols].fillna(train_df[common_cols].median())




    print(f"✅ Train shape: {train_df.shape}")
    if test_df is not None:
        print(f"✅ Test shape: {test_df.shape}")

    return train_df, test_df, stock_stats


In [ ]:
def temporal_train_val_split(df, val_frac=0.1, date_col='date_id', random_state=42):
    dates = np.sort(df[date_col].unique())
    n_val = max(1, int(len(dates) * val_frac))
    val_dates = dates[-n_val:]
    train_dates = dates[:-n_val]
    train_df = df[df[date_col].isin(train_dates)].reset_index(drop=True)
    val_df = df[df[date_col].isin(val_dates)].reset_index(drop=True)
    return train_df, val_df


train_df_full = pd.read_csv("train.csv")
print("Initial train shape:", train_df_full.shape)

train_df_full = reduce_mem_usage(train_df_full)
print("Reduced train shape:", train_df_full.shape)

# ---- STEP 5: Train/Validation Split ----
train_df, val_df = temporal_train_val_split(train_df_full, val_frac=0.1)
print("Reduced train shape:", train_df.shape, "Reduced validation shape:", val_df.shape)


train_df, val_df, stock_stats = process_dataset(train_df, val_df)

target_col = 'target'
feature_cols = [c for c in train_df.columns if c not in ['target', 'date_id', 'time_id', 'stock_id','row_id']]

X_train, y_train = train_df[feature_cols], train_df[target_col]
X_val, y_val = val_df[feature_cols], val_df[target_col]



Initial train shape: (5237980, 17)
Memory reduced by 374.65 MB (55.1%)
Reduced train shape: (5237980, 17)
Reduced train shape: (4709980, 17) Reduced validation shape: (528000, 17)
▶ Step 1: Basic feature engineering...
▶ Step 2: Time features...
▶ Step 3: Compute stock-level features from train and apply to all...
▶ Step 4: Market-wide features...
▶ Step 5: Lag features...
▶ Step 6: Extra features...
✅ Train shape: (4709980, 103)
✅ Test shape: (528000, 103)


In [6]:


def objective_lgb(trial):
    params = {
        'n_estimators': 400,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'max_depth': trial.suggest_int('max_depth', 9, 11),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'random_state': 42,
        'objective': 'regression_l1',
        'n_jobs': -1
    }

    model = LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='mae',
        callbacks=[early_stopping(50), log_evaluation(0)]
    )

    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=15, show_progress_bar=True)
print("Best LGB params:", study_lgb.best_trial.params)


[I 2025-10-20 11:37:16,063] A new study created in memory with name: no-name-e5b5e80f-9243-4250-8e64-e25e51430053


  0%|          | 0/15 [00:00<?, ?it/s]

[LightGBM] [Warning] feature_fraction is set=0.9294568657172472, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9294568657172472
[LightGBM] [Warning] bagging_fraction is set=0.7128491685751819, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7128491685751819
[LightGBM] [Warning] feature_fraction is set=0.9294568657172472, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9294568657172472
[LightGBM] [Warning] bagging_fraction is set=0.7128491685751819, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7128491685751819
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.157580 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22870
[LightGBM] [Info] Number of data points in the train set: 4709980, number of used features: 98
[LightGBM] [Warning] feature_fraction is s

In [7]:


def objective_xgb(trial):
    params = {
        'n_estimators': 400,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 6, 8),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        'random_state': 42,
        'tree_method': 'hist',
        'objective': 'reg:squarederror'
    }

    model = XGBRegressor(**params)


    try:
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='mae',
            early_stopping_rounds=30,
            verbose=False
        )
    except TypeError:
        model.fit(X_train, y_train, verbose=False)

    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=10, show_progress_bar=True)

print("✅ Best XGB params:", study_xgb.best_trial.params)
print("✅ Best validation MAE:", study_xgb.best_value)



[I 2025-10-20 11:56:29,484] A new study created in memory with name: no-name-174164e0-38ef-4125-8167-49df4c1ecb4c


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-20 11:57:37,170] Trial 0 finished with value: 5.876413345336914 and parameters: {'learning_rate': 0.09257685584930038, 'max_depth': 6, 'subsample': 0.9024186936422618, 'colsample_bytree': 0.9139275324869054, 'reg_lambda': 0.13738695701265477}. Best is trial 0 with value: 5.876413345336914.
[I 2025-10-20 11:59:02,125] Trial 1 finished with value: 5.869625568389893 and parameters: {'learning_rate': 0.0621146686645197, 'max_depth': 8, 'subsample': 0.7870470096841863, 'colsample_bytree': 0.8043740315844775, 'reg_lambda': 0.139712756884719}. Best is trial 1 with value: 5.869625568389893.
[I 2025-10-20 12:00:11,012] Trial 2 finished with value: 5.86167049407959 and parameters: {'learning_rate': 0.04709627535217298, 'max_depth': 6, 'subsample': 0.9968225734413377, 'colsample_bytree': 0.8183125092750367, 'reg_lambda': 0.28568992493391965}. Best is trial 2 with value: 5.86167049407959.
[I 2025-10-20 12:01:39,033] Trial 3 finished with value: 5.85768985748291 and parameters: {'learnin

In [8]:



def objective_hist(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 6, 8),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 200),
        'l2_regularization': trial.suggest_float('l2_regularization', 1e-4, 1.0, log=True),
        'max_iter': 500,
        'random_state': 42
    }

    model = HistGradientBoostingRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

study_hist = optuna.create_study(direction='minimize')
study_hist.optimize(objective_hist, n_trials=10, show_progress_bar=True)
print("Best HIST params:", study_hist.best_trial.params)


[I 2025-10-20 12:09:26,300] A new study created in memory with name: no-name-b9211da7-36e3-497d-8b68-06aa84ab4eb7


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2025-10-20 12:12:17,475] Trial 0 finished with value: 5.862231985502138 and parameters: {'learning_rate': 0.02684975123917489, 'max_depth': 6, 'min_samples_leaf': 88, 'l2_regularization': 0.0009250444715130107}. Best is trial 0 with value: 5.862231985502138.
[I 2025-10-20 12:14:59,726] Trial 1 finished with value: 5.866473502308536 and parameters: {'learning_rate': 0.01617462151244625, 'max_depth': 7, 'min_samples_leaf': 141, 'l2_regularization': 0.8826589181758928}. Best is trial 0 with value: 5.862231985502138.
[I 2025-10-20 12:17:10,378] Trial 2 finished with value: 5.865871213553847 and parameters: {'learning_rate': 0.057798050081713216, 'max_depth': 6, 'min_samples_leaf': 71, 'l2_regularization': 0.014360037120844209}. Best is trial 0 with value: 5.862231985502138.
[I 2025-10-20 12:19:07,764] Trial 3 finished with value: 5.861211859370273 and parameters: {'learning_rate': 0.06457496444743627, 'max_depth': 8, 'min_samples_leaf': 59, 'l2_regularization': 0.0001148559562452822}. B

In [ ]:
from sklearn.base import clone



def get_oof_preds(models, X, y, folds=5):
    oof = {name: np.zeros(len(X)) for name in models}
    preds_val = {name: None for name in models}
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    for tr_idx, val_idx in kf.split(X):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr = y.iloc[tr_idx]
        for name, model in models.items():
            m = clone(model)
            m.fit(X_tr, y_tr)
            oof[name][val_idx] = m.predict(X_va)
    oof_df = pd.DataFrame(oof)
    return oof_df


base_models = {
    "lgb": LGBMRegressor(**study_lgb.best_trial.params, n_estimators=1000),
    "xgb": XGBRegressor(**study_xgb.best_trial.params, n_estimators=1000),
    "hist": HistGradientBoostingRegressor(**study_hist.best_trial.params)
}






oof_preds = get_oof_preds(base_models, X_train, y_train, folds=5)
meta = RidgeCV(alphas=[0.1,1.0,10.0])
meta.fit(oof_preds, y_train)
# predict on validation
val_preds_base = pd.DataFrame({name: model.fit(X_train, y_train).predict(X_val) for name, model in base_models.items()})
ensemble_val_pred = meta.predict(val_preds_base)
print("Stacked MAE:", mean_absolute_error(y_val, ensemble_val_pred))


[LightGBM] [Warning] feature_fraction is set=0.986385487639305, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.986385487639305
[LightGBM] [Warning] bagging_fraction is set=0.810473184318834, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810473184318834
[LightGBM] [Warning] feature_fraction is set=0.986385487639305, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.986385487639305
[LightGBM] [Warning] bagging_fraction is set=0.810473184318834, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810473184318834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.136063 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22869
[LightGBM] [Info] Number of data points in the train set: 3767984, number of used features: 98
[LightGBM] [Info] Start training from score -0.045

In [10]:
import joblib

# Save your fitted artifacts
joblib.dump(stock_stats, "stock_stats.pkl")
joblib.dump(base_models, "base_models.pkl")
joblib.dump(meta, "meta_model.pkl")


['meta_model.pkl']

In [ ]:
# def compute_stock_embeddings(df):
#     """Compute per-stock summary statistics (no future leakage)."""
#     stock_stats = df.groupby('stock_id').agg(
#         wap_mean=('wap', 'mean'),
#         wap_std=('wap', 'std'),
#         wap_skew=('wap', lambda x: x.skew()),
#         spread_mean=('spread', 'mean'),
#         spread_std=('spread', 'std'),
#         matched_mean=('matched_size', 'mean'),
#         imbalance_mean=('imbalance_size', 'mean'),
#         imbalance_std=('imbalance_size', 'std'),
#         buy_sell_flag_mean=('imbalance_buy_sell_flag', 'mean'),
#         liquidity_mean=('matched_size', lambda x: x.mean() / (x.mean() + 1e-6))
#     ).fillna(0)
#     return stock_stats.reset_index()



# def fit_kmeans_and_neighbors(stock_stats, n_clusters=20, n_neighbors=6, random_state=42):
#     X = stock_stats.drop(columns=['stock_id']).values
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(X)

#     kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init='auto')
#     kmeans.fit(X_scaled)

#     # nearest neighbor model
#     nn = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')
#     nn.fit(X_scaled)

#     # attach results
#     stock_stats['cluster'] = kmeans.labels_
#     stock_stats['embedding'] = list(X_scaled)
#     return scaler, kmeans, nn, stock_stats, X_scaled


# def get_similar_and_opposite(stock_stats, X_scaled, n_similar=2):
#     stock_ids = stock_stats['stock_id'].values
#     dist = pairwise_distances(X_scaled, metric='euclidean')

#     results = []
#     for i, sid in enumerate(stock_ids):
#         order = np.argsort(dist[i])
#         similar_ids = stock_ids[order[1:n_similar+1]]  # skip self (index 0)
        
#         # create dictionary with dynamic keys
#         row = {'stock_id': sid}
#         for j, sim_id in enumerate(similar_ids):
#             row[f'similar_stock{j+1}'] = sim_id
#         results.append(row)
    
#     # convert list of dicts → DataFrame
#     return pd.DataFrame(results)



# # 1. Compute stock-level embeddings
# stock_stats = compute_stock_embeddings(engineered_df)   # train_df = your training data DataFrame

# # 2. Fit clustering and neighbors
# scaler, kmeans, nn, stock_stats, X_scaled = fit_kmeans_and_neighbors(stock_stats,
#                                                                      n_clusters=20,
#                                                                      n_neighbors=6,
#                                                                      random_state=42)

# # 3. Get similar and opposite stock IDs for each stock
# pairs_df = get_similar_and_opposite(stock_stats, X_scaled,
#                                     n_similar=2)


# df_with_neighbors = engineered_df.merge(pairs_df, on='stock_id', how='left')
# df_with_neighbors



,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,liquidity_ratio,bid_pressure,volatility_proxy,volume,size_imbalance,wap_mean_all,wap_std_all,wap_relative_to_market,similar_stock1,similar_stock2
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,NaN,NaN,0.999812,...,0.807945,0.877170,NaN,69144.531250,7.141326,1.000000,0.000000,1.000000,137,186
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,NaN,NaN,0.999896,...,0.907894,0.135625,NaN,23838.128906,0.156905,1.000000,0.000000,1.000000,118,119
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,NaN,NaN,0.999403,...,0.857283,0.666468,NaN,56951.000000,1.998210,1.000000,0.000000,1.000000,94,146
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,NaN,NaN,0.999999,...,0.606774,0.004830,NaN,481357.312500,0.004853,1.000000,0.000000,1.000000,123,37
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,NaN,NaN,0.999394,...,0.975555,0.974343,NaN,16919.638672,37.976360,1.000000,0.000000,1.000000,186,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2.440723e+06,-1,1.000317,28280362.00,0.999734,0.999734,1.000317,...,0.920552,0.091608,0.000000,352119.437500,0.100847,0.999098,0.002293,1.001231,166,187
5237976,196,480,540,3.495105e+05,-1,1.000643,9187699.00,1.000129,1.000386,1.000643,...,0.963353,0.687127,-0.000257,298501.468750,2.196184,0.999098,0.002293,1.001722,182,32
5237977,197,480,540,0.000000e+00,0,0.995789,12725436.00,0.995789,0.995789,0.995789,...,1.000000,0.085306,0.000000,196828.968750,0.093262,0.999098,0.002293,0.996696,194,128
5237978,198,480,540,1.000899e+06,1,0.999210,94773272.00,0.999210,0.999210,0.998970,...,0.989549,0.157923,0.000000,795524.750000,0.187540,0.999098,0.002293,0.999910,160,163
